Importing libraries

In [14]:
import os
import gc
import re
import string
import operator
import numpy as np
import pandas as pd
pd.set_option('display.max_rows', 500)
pd.set_option('display.max_columns', 500)
pd.set_option('display.width', 1000)
import matplotlib
import matplotlib.pyplot as plt
import tokenization
from wordcloud import STOPWORDS
import seaborn as sns
from nltk.corpus import stopwords
from nltk.util import ngrams
from collections import defaultdict
from collections import  Counter
plt.style.use('ggplot')
stop=set(stopwords.words('english'))
from nltk.tokenize import word_tokenize
import gensim
from tqdm import tqdm
from sklearn.model_selection import train_test_split
from sklearn.model_selection import StratifiedKFold, StratifiedShuffleSplit
from sklearn.metrics import precision_score, recall_score, f1_score
from sklearn.feature_extraction.text import CountVectorizer
import tensorflow as tf
import tensorflow_hub as hub
from tensorflow import keras
from keras.preprocessing.text import Tokenizer
from keras_preprocessing.sequence import pad_sequences
from tensorflow.keras.optimizers import SGD, Adam
from tensorflow.keras.layers import Dense, Input, Dropout, GlobalAveragePooling1D
from tensorflow.keras.models import Model, Sequential
from tensorflow.keras.callbacks import ModelCheckpoint, EarlyStopping, Callback
from keras.models import Sequential
from keras.layers import Embedding,LSTM,Dense,SpatialDropout1D
from keras.initializers import Constant
from tensorflow.keras.optimizers import Adam
SEED = 1337

Importing datasets

In [15]:
train= pd.read_csv("/content/drive/MyDrive/train.csv")
test=pd.read_csv("/content/drive/MyDrive/test.csv")
train.head(3)

,id,keyword,location,text,target
0,1,NaN,NaN,Our Deeds are the Reason of this #earthquake M...,1
1,4,NaN,NaN,Forest fire near La Ronge Sask. Canada,1
2,5,NaN,NaN,All residents asked to 'shelter in place' are ...,1


In [16]:
print('There are {} rows and {} columns in train'.format(train.shape[0],train.shape[1]))
print('There are {} rows and {} columns in train'.format(test.shape[0],test.shape[1]))

There are 7613 rows and 5 columns in train
There are 3263 rows and 4 columns in train


Data Cleaning

In [17]:
df=pd.concat([train,test])
df.shape

(10876, 5)

In [18]:
example="New competition launched :https://www.kaggle.com/c/nlp-getting-started"
def remove_URL(text):
    url = re.compile(r'https?://\S+|www\.\S+')
    return url.sub(r'',text)
remove_URL(example)

'New competition launched :'

In [19]:
df['text']=df['text'].apply(lambda x : remove_URL(x))

In [20]:
example = """<div>
<h1>Real or Fake</h1>
<p>Kaggle </p>
<a href="https://www.kaggle.com/c/nlp-getting-started">getting started</a>
</div>"""
def remove_html(text):
    html=re.compile(r'<.*?>')
    return html.sub(r'',text)
print(remove_html(example))


Real or Fake
Kaggle 
getting started



In [21]:
df['text']=df['text'].apply(lambda x : remove_html(x))

In [22]:
def remove_emoji(text):
    emoji_pattern = re.compile("["
                           u"\U0001F600-\U0001F64F"  
                           u"\U0001F300-\U0001F5FF"  
                           u"\U0001F680-\U0001F6FF"  
                           u"\U0001F1E0-\U0001F1FF" 
                           u"\U00002702-\U000027B0"
                           u"\U000024C2-\U0001F251"
                           "]+", flags=re.UNICODE)
    return emoji_pattern.sub(r'', text)
remove_emoji("Omg another Earthquake 😔😔")

'Omg another Earthquake '

In [23]:
df['text']=df['text'].apply(lambda x: remove_emoji(x))

In [24]:
def remove_punct(text):
    table=str.maketrans('','',string.punctuation)
    return text.translate(table)
example="I am a #king"
print(remove_punct(example))

I am a king


In [25]:
df['text']=df['text'].apply(lambda x : remove_punct(x))

In [28]:
from spellchecker import SpellChecker
spell = SpellChecker()
def correct_spellings(text):
    corrected_text = []
    misspelled_words = spell.unknown(text.split())
    for word in text.split():
        if word in misspelled_words:
            corrected_text.append(spell.correction(word))
        else:
            corrected_text.append(word)
    return " ".join(corrected_text)    
text = "corect me plese"
correct_spellings(text)

'correct me plese'

GloVe for Vectorization

In [32]:
def create_corpus(df):
    corpus=[]
    for tweet in tqdm(df['text']):
        words=[word.lower() for word in word_tokenize(tweet) if((word.isalpha()==1) & (word not in stop))]
        corpus.append(words)
    return corpus
corpus=create_corpus(df)

100%|██████████| 10876/10876 [00:01<00:00, 6903.09it/s]


In [33]:
embedding_dict={}
with open('/content/drive/MyDrive/glove.6B.100d.txt','r') as f:
    for line in f:
        values=line.split()
        word=values[0]
        vectors=np.asarray(values[1:],'float32')
        embedding_dict[word]=vectors
f.close()

In [34]:
MAX_LEN=50
tokenizer_obj=Tokenizer()
tokenizer_obj.fit_on_texts(corpus)
sequences=tokenizer_obj.texts_to_sequences(corpus)
tweet_pad=pad_sequences(sequences,maxlen=MAX_LEN,truncating='post',padding='post')

In [35]:
word_index=tokenizer_obj.word_index
print('Number of unique words:',len(word_index))

Number of unique words: 20342


In [36]:
num_words=len(word_index)+1
embedding_matrix=np.zeros((num_words,100))
for word,i in tqdm(word_index.items()):
    if i > num_words:
        continue
    emb_vec=embedding_dict.get(word)
    if emb_vec is not None:
        embedding_matrix[i]=emb_vec  

100%|██████████| 20342/20342 [00:00<00:00, 312791.80it/s]


Baseline Model

In [37]:
model=Sequential()
embedding=Embedding(num_words,100,embeddings_initializer=Constant(embedding_matrix),
                   input_length=MAX_LEN,trainable=False)
model.add(embedding)
model.add(SpatialDropout1D(0.2))
model.add(LSTM(64, dropout=0.2, recurrent_dropout=0.2))
model.add(Dense(1, activation='sigmoid'))
optimzer=Adam(learning_rate=1e-5)
model.compile(loss='binary_crossentropy',optimizer=optimzer,metrics=['accuracy'])

In [38]:
model.summary()

Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 embedding (Embedding)       (None, 50, 100)           2034300   
                                                                 
 spatial_dropout1d (SpatialD  (None, 50, 100)          0         
 ropout1D)                                                       
                                                                 
 lstm (LSTM)                 (None, 64)                42240     
                                                                 
 dense (Dense)               (None, 1)                 65        
                                                                 
Total params: 2,076,605
Trainable params: 42,305
Non-trainable params: 2,034,300
_________________________________________________________________


In [40]:
train10=tweet_pad[:train.shape[0]]
test=tweet_pad[train.shape[0]:]

In [41]:
X_train10,X_test,y_train10,y_test=train_test_split(train10,train['target'].values,test_size=0.15)
print('Shape of train',X_train10.shape)
print("Shape of Validation ",X_test.shape)

Shape of train (6471, 50)
Shape of Validation  (1142, 50)


In [42]:
history=model.fit(X_train10,y_train10,batch_size=4,epochs=15,validation_data=(X_test,y_test),verbose=2)

Epoch 1/15
1618/1618 - 107s - loss: 0.6759 - accuracy: 0.5973 - val_loss: 0.5864 - val_accuracy: 0.7478 - 107s/epoch - 66ms/step
Epoch 2/15
1618/1618 - 98s - loss: 0.5612 - accuracy: 0.7419 - val_loss: 0.5498 - val_accuracy: 0.7469 - 98s/epoch - 60ms/step
Epoch 3/15
1618/1618 - 94s - loss: 0.5505 - accuracy: 0.7489 - val_loss: 0.5438 - val_accuracy: 0.7539 - 94s/epoch - 58ms/step
Epoch 4/15
1618/1618 - 91s - loss: 0.5344 - accuracy: 0.7558 - val_loss: 0.5292 - val_accuracy: 0.7548 - 91s/epoch - 56ms/step
Epoch 5/15
1618/1618 - 92s - loss: 0.5255 - accuracy: 0.7602 - val_loss: 0.5227 - val_accuracy: 0.7557 - 92s/epoch - 57ms/step
Epoch 6/15
1618/1618 - 91s - loss: 0.5127 - accuracy: 0.7716 - val_loss: 0.5176 - val_accuracy: 0.7627 - 91s/epoch - 56ms/step
Epoch 7/15
1618/1618 - 88s - loss: 0.5092 - accuracy: 0.7699 - val_loss: 0.5098 - val_accuracy: 0.7653 - 88s/epoch - 54ms/step
Epoch 8/15
1618/1618 - 88s - loss: 0.5009 - accuracy: 0.7735 - val_loss: 0.5040 - val_accuracy: 0.7715 - 88s/

Making submission

In [43]:
sample_sub=pd.read_csv('/content/drive/MyDrive/sample_submission.csv')

In [44]:
y_pre=model.predict(test)
y_pre=np.round(y_pre).astype(int).reshape(3263)
sub=pd.DataFrame({'id':sample_sub['id'].values.tolist(),'target':y_pre})
sub.to_csv('submission.csv',index=False)

In [46]:
sub.head()

,id,target
0,0,1
1,2,1
2,3,1
3,9,1
4,11,1
